In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install -qqq ftfy
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

## for opening, manipulating, and saving many different image file f
from PIL import Image

import random 

## for plotting
import matplotlib.pyplot as plt

## for processing
import nltk
nltk.download('omw-1.4')
import re
import ftfy
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger') 

#for expanding contracted words
import sys  
!{sys.executable} -m pip install contractions
import contractions

## WordCloud - Python linrary for creating image wordclouds
from wordcloud import WordCloud
from nltk import pos_tag ## For Parts of Speech tagging

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.9 MB/s eta 0:00:00


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 33.3 MB/s eta 0:00:00


SELECTING DATASETS

In [ ]:
ds_all_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Dataset/depression_detection_Modified.csv")

In [ ]:
ds_all_train.shape

(39152, 2)

In [ ]:
ds_train_dep = ds_all_train[ds_all_train['label']==1]

In [ ]:
ds_train_notdep = ds_all_train[ds_all_train['label']==0]

In [ ]:
ds_all = ds_train_dep.append(ds_train_notdep)

In [ ]:
ds_all

,text,label
0,Open discussion. Between the Transfer Portal a...,1
1,Plenty of things are changing in my life and t...,1
2,I feel a little hopeless. Anyone else? #hopele...,1
3,"Which is more healthy? Hope, or hopelessness? ...",1
4,So someone tell me how do I get over #HOPELESS...,1
...,...,...
39143,I am not in depression,0
39144,I am not sad,0
39147,He is not depressed but happy,0
39150,He is not in depression,0


DATASET NOISE CLEANING (PRE-PROCESSING)

In [ ]:
def expandContracted(sentence):
  expanded_words = []   
  for word in sentence.split():
    # using contractions.fix to expand the shortened words
    expanded_words.append(contractions.fix(word))  
  res = ' '.join(expanded_words)
  return res

In [ ]:
# LEMMATIZATION WITH POS TAG
# POS_TAGGER_FUNCTION : TYPE 1
def pos_tagger(nltk_tag):
	if nltk_tag.startswith('J'):
		return wordnet.ADJ
	elif nltk_tag.startswith('V'):
		return wordnet.VERB
	elif nltk_tag.startswith('N'):
		return wordnet.NOUN
	elif nltk_tag.startswith('R'):
		return wordnet.ADV
	else:		
		return None

def lemmatize_func(text):
  lemmatizer = WordNetLemmatizer()
  # tokenize the sentence and find the POS tag for each token
  pos_tagged = nltk.pos_tag(nltk.word_tokenize(text))

  wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
  lemmatized_sentence = []
  for word, tag in wordnet_tagged:
    if tag is None:
      lemmatized_sentence.append(word)
    else:
      lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
  lemmatized_sentence = " ".join(lemmatized_sentence)

  return lemmatized_sentence

In [ ]:
def Negation(sentence):	
  '''
  Input: Tokenized sentence (List of words)
  Output: Tokenized sentence with negation handled (List of words)
  '''
  temp = int(0)
  sentence = nltk.word_tokenize(sentence)
  for i in range(len(sentence)):
      if sentence[i-1] in ['not',"n't"]:
          antonyms = []
          for syn in wordnet.synsets(sentence[i]):
              syns = wordnet.synsets(sentence[i])
              w1 = syns[0].name()
              temp = 0
              for l in syn.lemmas():
                  if l.antonyms():
                      antonyms.append(l.antonyms()[0].name())
              max_dissimilarity = 0
              for ant in antonyms:
                  syns = wordnet.synsets(ant)
                  w2 = syns[0].name()
                  syns = wordnet.synsets(sentence[i])
                  w1 = syns[0].name()
                  word1 = wordnet.synset(w1)
                  word2 = wordnet.synset(w2)
                  if isinstance(word1.wup_similarity(word2), float) or isinstance(word1.wup_similarity(word2), int):
                      temp = 1 - word1.wup_similarity(word2)
                  if temp>max_dissimilarity:
                      max_dissimilarity = temp
                      antonym_max = ant
                      sentence[i] = antonym_max
                      sentence[i-1] = ''
  res=""
  while '' in sentence:
      sentence.remove('')
    
  for x in sentence:
    res += x + " "
  return res

In [ ]:
# PRE-PROCESSING FUNCTION
def cleaning_process(input_tweets, input_labels):
  cleaned_tweets = []
  cleaned_labels = []
  for (a_tweet, a_label) in zip(input_tweets, input_labels):

    #convert to lowercase
    a_tweet = a_tweet.lower()
    
    # if url links then don't append to avoid news articles
    # also check tweet length, save those > 5 
    if re.match("(\w+:\/\/\S+)", a_tweet) == None and len(a_tweet) > 5:
      #remove @mention
      a_tweet = re.sub(r"(?:\@|https?\://)\S+", "", a_tweet)

      #expand contraction
      a_tweet = expandContracted(a_tweet)

      a_tweet = Negation(a_tweet)

      #remove punctuation
      a_tweet = ' '.join(re.sub("([^0-9A-Za-z \t])", " ", a_tweet).split())

      #remove hashtag, @mention, HTML tags and image URLs
      a_tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(\#[A-Za-z0-9]+)|(<.>)|(pic\.twitter\.com\/.*)", " ", a_tweet).split())
      
      #remove numbers
      a_tweet = re.sub(r'\d+', '', a_tweet)

      #remove urls
      a_tweet = re.sub(r'https?://\S+|www\.\S+', '', a_tweet)

      #fix weirdly encoded texts (Unicode correction)
      a_tweet = ftfy.fix_text(a_tweet)

      #stop words
      stop_words = set(stopwords.words('english'))
      word_tokens = nltk.word_tokenize(a_tweet)
      a_tweet = [word for word in word_tokens if not word in stop_words or word == "not"]
      a_tweet = ' '.join(a_tweet) # join words with a space in between them

      #lemmatization
      a_tweet = lemmatize_func(a_tweet)

      cleaned_tweets.append(a_tweet)
      cleaned_labels.append(a_label)
  return (cleaned_tweets, cleaned_labels)

CLEANING OF EYE DATASET

In [ ]:
ds_text_all_arr = [x for x in ds_all['text']]
ds_label_all_arr = [x for x in ds_all['label']]
(cleaned_ds_text_all_arr, cleaned_ds_labels_all_arr) = cleaning_process(ds_text_all_arr, ds_label_all_arr)

In [ ]:
print(ds_text_all_arr[:10])

['Open discussion. Between the Transfer Portal and the NIL, will the @NCAA become obsolete as an organization and governing body? @zlancaster91 @RAllenGoPokes #Hopelessness #GoPokes #LoyalandTrue', "Plenty of things are changing in my life and the lives of those around me. There is one thing that doesn't change, my #hopelessness.", 'I feel a little hopeless. Anyone else? #hopelessness', 'Which is more healthy? Hope, or hopelessness? #hope #Hopelessness #Mentalhealth', "So someone tell me how do I get over #HOPELESSNESS? \nI live in a world of #poverty surrounded by #PoorPeople.\nPeople help us that are not much better off than we are.\nIf not for my son's inability to care for himself I would likely give into the Hopelessness &amp; darkness.", 'No parent deserves to experience the Indian legal system. #hopelessness', 'Being in a #union also looks a lot like being #alone.\n\nIt can feel like there’s no worse place to be sometimes.\n\n#hopelessness #unions #workersrights', 'I am so glad 

In [ ]:
print(cleaned_ds_text_all_arr[:10])

['open discussion transfer portal nil become obsolete organization govern body hopelessness gopokes loyalandtrue', 'plenty thing change life life around one thing stay hopelessness', 'feel little hopeless anyone else hopelessness', 'healthy hope hopelessness hope hopelessness mentalhealth', 'someone tell get hopelessness live world poverty surround poorpeople people help us little well not son inability care would likely give hopelessness amp darkness', 'parent deserve experience indian legal system hopelessness', 'union also look lot like alone feel like bad place sometimes hopelessness union workersrights', 'glad tackle huge healthcare professional shortage face sadden not really live post pandemic world burnout hopelessness stress palpable every level department system', 'know someone depressed please resolve never ask depression not straightforward response bad situation depression like weather try understand blackness lethargy hopelessness loneliness go', 'grateful heart seegod fi

BAG OF WORDS

In [ ]:
from collections import Counter
from itertools import chain
from nltk.util import ngrams
from nltk.util import bigrams

def bow_for_line(corpus, top_words):
  bag_of_words = []
  for doc in corpus:
    unigram_words = doc
    bigram_words = list(bigrams(doc))
    trigram_words = list(ngrams(doc, 3))
    
    word_counts = Counter(doc)
    bigram_word_counts = Counter(bigram_words)
    trigram_word_counts = Counter(trigram_words)

    word_count=dict(word_counts)
    word_count.update(dict(bigram_word_counts))
    word_count.update(dict(trigram_word_counts))

    dict_words = dict(word_count)
    line_words = Counter(dict_words)

    row = [line_words[word] if word in line_words else 0 for word in top_words]
    bag_of_words.append(row)

  return bag_of_words

def bow(data_train):
  # Preprocess and tokenize the corpus
  corpus = [nltk.word_tokenize(doc.lower()) for doc in data_train]

  # Calculate the frequency of each word in the corpus
  word_counts = Counter(chain.from_iterable(corpus))

  bigram_words = []
  for doc in corpus:
    words = list(bigrams(doc))
    bigram_words.append(words)

  # Calculate the frequency of each bi and trigram words in the corpus
  bigram_word_counts = Counter(chain.from_iterable(bigram_words))

  trigram_words = []
  for doc in corpus:
    words = list(ngrams(doc, 3))
    trigram_words.append(words)

  trigram_word_counts = Counter(chain.from_iterable(trigram_words))

  word_count=dict(word_counts)
  word_count.update(dict(bigram_word_counts))
  word_count.update(dict(trigram_word_counts))

  word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
  dict_words = dict(word_count)
  final_words = Counter(dict_words)

  # # Select the top k most frequent words as columns
  top_words = [word for word, count in final_words.most_common(5000)]

  # # # Create the bag of words matrix
  bag_of_words = bow_for_line(corpus, top_words)
  
  return bag_of_words, top_words

BOW for EYE

In [ ]:
bag_of_words, top_words = bow(cleaned_ds_text_all_arr)

print(len(bag_of_words[0]))
print(len(top_words))

5000
5000


In [ ]:
column_words = []
for word in top_words:
  if isinstance(word, tuple):
    column_words.append(" ".join(word))
  else:
    column_words.append(word)

In [ ]:
df = pd.DataFrame (bag_of_words, columns = column_words)
df.head()

,mentalhealth,anxiety,not,depression,quot,sad,get,go,love,like,...,amp tweet,time quot,quot god,haha love,one quot,followfriday great,lol well,shaundiviney shaundiviney,quot really,thanks followfriday
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X = df
y = cleaned_ds_labels_all_arr
print(len(X), len(y))

39152 39152


10 - FOLD CROSS VALIDATION FOR MODELS

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=10, shuffle=True)
splits=list(kf.split(X,y))

LOGISTIC REGRESSION

In [ ]:
LogReg_scores_eye = []
LogReg_scores = []

f1_scores_eye = []
precision_scores_eye = []
recall_scores_eye = []

for k in range(10):
  train_indices, test_indices = splits[k]
  X_train = np.array(X)[train_indices.astype(int)]
  X_test = np.array(X)[test_indices.astype(int)]

  y_train = np.array(y)[train_indices.astype(int)]
  y_test = np.array(y)[test_indices.astype(int)]

  model_Log = LogisticRegression()
  model_Log.fit(X_train, y_train)
  y_pred = model_Log.predict(X_test)
  
  print("Fold - {} completed".format(k))

  #Eye testing
  LogReg_scores_eye.append(model_Log.score(X_test, y_test))
  y_pred_eye = model_Log.predict(X_test)
  f1_scores_eye.append(f1_score(y_test, y_pred_eye, average='macro'))
  precision_scores_eye.append(precision_score(y_test, y_pred_eye, average='macro'))
  recall_scores_eye.append(recall_score(y_test, y_pred_eye, average='macro'))

print('\n\n10 Fold Cross-Validation accuracy for training and testing with EYE dataset:')
print('CV accuracy LogReg: %.4f' %(np.mean(LogReg_scores_eye)*100 + np.std(LogReg_scores_eye)*100))
print('CV F1 score: %.4f' %(np.mean(f1_scores_eye)*100 + np.std(f1_scores_eye)*100))
print('CV Precision: %.4f' %(np.mean(precision_scores_eye)*100 + np.std(precision_scores_eye)*100))
print('CV Recall: %.4f' %(np.mean(recall_scores_eye)*100 + np.std(recall_scores_eye)*100))

LogReg_scores.append(np.mean(LogReg_scores_eye)*100 + np.std(LogReg_scores_eye)*100)
LogReg_scores.append(np.mean(precision_scores_eye)*100 + np.std(precision_scores_eye)*100)
LogReg_scores.append(np.mean(recall_scores_eye)*100 + np.std(recall_scores_eye)*100)
LogReg_scores.append(np.mean(f1_scores_eye)*100 + np.std(f1_scores_eye)*100)

Fold - 0 completed
Fold - 1 completed
Fold - 2 completed
Fold - 3 completed
Fold - 4 completed
Fold - 5 completed
Fold - 6 completed
Fold - 7 completed
Fold - 8 completed
Fold - 9 completed


10 Fold Cross-Validation accuracy for training and testing with EYE dataset:
CV accuracy LogReg: 99.9043
CV F1 score: 99.9038
CV Precision: 99.9004
CV Recall: 99.9075


NO CV MODEL

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model_Log.predict(X_test)

In [ ]:
print(accuracy_score(y_test, y_pred))

0.9988390991409334


SAVE MODEL

In [ ]:
import pickle

In [ ]:
filename = 'Final_Model.sav'
pickle.dump(model, open(filename, 'wb'))

SAVE TOP WORDS

In [ ]:
with open(r'top_words.txt', 'w') as fp:
    fp.write("\n".join(str(item) for item in top_words))